In [9]:
import csv
import json
from datetime import datetime, timedelta, timezone

# Read CSV dataset
with open("data.csv", "r") as f:
    reader = csv.DictReader(f)
    rows = list(reader)

total = len(rows)
delivered = 0
rto = 0
risk_orders = 0

# Current IST time
now = datetime.now(timezone(timedelta(hours=5, minutes=30)))
seven_days_ago = now - timedelta(days=7)

for row in rows:
    status = row["current_status"].strip().upper()
    created_at = datetime.fromisoformat(row["created_at"].replace("Z", "+05:30"))

    if status == "DELIVERED":
        delivered += 1
    elif "RTO" in status:   # handles "RTO_DELIVERED" or "RTO"
        rto += 1

    # Order at risk → placed in last 7 days and not delivered
    if created_at >= seven_days_ago and status != "DELIVERED":
        risk_orders += 1

# KPIs
delivered_percent = round((delivered / total) * 100, 2) if total else 0
rto_percent = round((rto / total) * 100, 2) if total else 0
drr = round(delivered / rto, 3) if rto > 0 else 0

# Build JSON
submission = {
    "submitted_at_ist": now.isoformat(),
    "kpi": {
        "delivered_percent": delivered_percent,
        "rto_percent": rto_percent,
        "orders_at_risk": risk_orders,
        "drr": drr
    },
    "rules_thresholds": {
        "delivered_percent": 85,
        "rto_percent": 8,
        "orders_at_risk": 20
    },
    "rules_triggers": []
}

with open("submission.json", "w") as f:
    json.dump(submission, f, indent=2)

print("✅ submission.json generated")


✅ submission.json generated
